# Feature Visualization

In this notebook, we try to use t-sne method to visualize the features of images extracted from the pre-trained Inception v3 model.

In [117]:
import cv2
import sqlite3
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from glob import glob
from collections import Counter
from os.path import join

## Sample Compounds

For visualization, we can just choose a subsample of compounds.

In [109]:
df = pd.read_csv("./data/test/meta_data/profiles/mean_well_profiles.csv")
print(df.shape)
df.head(10)

(384, 1800)


,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_ASSAY_WELL_ROLE,Metadata_broad_sample,Metadata_mmoles_per_liter,Metadata_solvent,Metadata_pert_id,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_3_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_3_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_3_0,Nuclei_Texture_Variance_RNA_5_0
0,24278,a01,24278,H-BIOA-007-3,a01,treated,BRD-K78364995-236-03-5,2.090060,DMSO,BRD-K78364995,...,3.065597,1.525995,1.596880,1.557685,1.633298,1.627728,1.618267,2.582515,2.474853,2.528092
1,24278,a02,24278,H-BIOA-007-3,a02,treated,BRD-K78414110-001-02-8,5.000000,DMSO,BRD-K78414110,...,2.947674,1.595292,1.681501,1.639656,1.438692,1.497101,1.473400,2.402188,2.443030,2.527604
2,24278,a03,24278,H-BIOA-007-3,a03,treated,BRD-K78485176-001-02-9,5.000000,DMSO,BRD-K78485176,...,2.959839,1.573649,1.641233,1.630217,1.470714,1.545823,1.499599,2.400266,2.457235,2.536708
3,24278,a04,24278,H-BIOA-007-3,a04,treated,BRD-K78496197-001-01-3,5.000000,DMSO,BRD-K78496197,...,3.323827,1.627224,1.800667,1.699291,1.419890,1.578673,1.528876,2.601877,2.439709,2.506089
4,24278,a05,24278,H-BIOA-007-3,a05,treated,BRD-K78599730-001-02-6,5.000000,DMSO,BRD-K78599730,...,3.344600,1.772216,1.913539,1.881990,1.292479,1.424874,1.378594,2.433934,2.356783,2.422295
5,24278,a06,24278,H-BIOA-007-3,a06,treated,BRD-K78612426-001-02-6,5.000000,DMSO,BRD-K78612426,...,3.499219,1.746578,1.896714,1.836252,1.604872,1.700492,1.687575,2.498050,2.398161,2.474145
6,24278,a07,24278,H-BIOA-007-3,a07,treated,BRD-K78633253-001-01-2,5.000000,DMSO,BRD-K78633253,...,3.715294,1.823429,2.092295,2.010445,1.585145,1.758397,1.675881,2.509877,2.409396,2.488828
7,24278,a08,24278,H-BIOA-007-3,a08,treated,BRD-K78637815-001-01-4,5.000000,DMSO,BRD-K78637815,...,3.360001,1.693696,1.795996,1.783097,1.587473,1.714071,1.649360,2.390896,2.383596,2.445147
8,24278,a09,24278,H-BIOA-007-3,a09,treated,BRD-K78643075-001-03-3,2.379167,DMSO,BRD-K78643075,...,3.559220,1.857709,1.973541,1.909170,1.654707,1.760128,1.705003,2.320583,2.282689,2.343038
9,24278,a10,24278,H-BIOA-007-3,a10,treated,BRD-K78692225-001-11-2,5.000000,DMSO,BRD-K78692225,...,3.543376,1.810797,1.847711,1.791665,1.531595,1.699469,1.603155,2.381528,2.369899,2.450524


In this plate, 64 wells are treated using DMSO (control group). Other wells all have different chemical compounds. We can just randomly sample 3 other compounds.

In [3]:
compounds = set(df['Metadata_broad_sample'])
sample = random.sample(compounds, 3)
df[df['Metadata_broad_sample'].isin(sample)]

,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_ASSAY_WELL_ROLE,Metadata_broad_sample,Metadata_mmoles_per_liter,Metadata_solvent,Metadata_pert_id,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_3_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_3_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_3_0,Nuclei_Texture_Variance_RNA_5_0
14,24278,a15,24278,H-BIOA-007-3,a15,treated,BRD-K78844995-001-01-8,5.0,DMSO,BRD-K78844995,...,3.903239,1.837795,2.029369,1.976614,1.689389,1.866387,1.731690,2.595526,2.477372,2.514824
227,24278,j12,24278,H-BIOA-007-3,j12,treated,BRD-K91301684-003-02-2,5.0,DMSO,BRD-K91301684,...,3.426926,1.861431,2.092776,2.009441,1.703690,1.901646,1.839830,2.341294,2.456272,2.469435
381,24278,p22,24278,H-BIOA-007-3,p22,treated,BRD-K98281970-001-02-6,5.0,DMSO,BRD-K98281970,...,3.706245,2.040592,2.203903,2.137545,1.772094,2.030643,1.948805,2.452169,2.469453,2.466105


Then we use the [python scrip](./script/image_processing.py) to crop all single cell images from those selected wells.

For each cell, we have two outputs (with RGB channels merged from channels 1, 2, 3 and channels 4, 5).

In [150]:
def compute_entropy(img_name):
    """
    Compute the entropy function (like entropy() in Matlab)
    """
    
    img = cv2.imread(img_name, 0)
    cur_hist = cv2.calcHist([img],[0], None, [256], [0,256]).flatten()

    # Remove zeroes
    hist_no_zero = np.array([i for i in cur_hist if i != 0])

    # Normalize the hist so it sums to 1
    hist_no_zero = hist_no_zero / np.size(img)

    # Compute the entropy
    log_2 = np.log(hist_no_zero) / np.log(2)
    entropy = -1 * np.dot(hist_no_zero, log_2)
    return entropy
    
    
def compute_entropy_dir(input_dir):
    """ 
    Wrapper function for compute_entropy() in dir level.
    Return two lists [image_names], [entropy] with the same index.
    """
    
    images, entropies = [], []
    for image in glob(join(input_dir, "*.tif")):
        images.append(image)
        entropies.append(compute_entropy(image))

    return images, entropies

def plot_images(stack, row, column):
    """
    Plot all the images in the row matrix of images in the 
    given format. Automatically add padding to the end.
    """
    image_size = stack.shape[0]
    image = np.zeros((row*image_size, column*image_size, 3)).astype(np.uint16)
    for i in range(int(stack.shape[1]/image_size)):
        r, c = i // column, i % column
        image[image_size*r:image_size*(r+1),
              image_size*c:image_size*(c+1), :] = stack[:, image_size*i:image_size*(i+1), :]
    return image

In [151]:
# Compute and srot the entropy
a13_sorted_entropy = sorted(zip(*compute_entropy_dir("train/24278_a13/c123")), key=lambda x: x[1])
a15_sorted_entropy = sorted(zip(*compute_entropy_dir("train/24278_a15/c123")), key=lambda x: x[1])
a13_sorted_entropy_2 = sorted(zip(*compute_entropy_dir("train/24278_a13/c45")), key=lambda x: x[1])
a15_sorted_entropy_2 = sorted(zip(*compute_entropy_dir("train/24278_a15/c45")), key=lambda x: x[1])

In [156]:
# Plot the images
temp_list = []
for r in a13_sorted_entropy:
    img = cv2.imread(r[0].replace("train", "out").replace('tif', 'png'), -1)
    temp_list.append(cv2.resize(img, (100, 100)))
img_stack = np.hstack(temp_list)
img_total = plot_images(img_stack, 36, 10)
cv2.imwrite("./plots/a13_ch123.png", img_total)

temp_list = []
for r in a13_sorted_entropy_2:
    img = cv2.imread(r[0].replace("train", "out").replace('tif', 'png'), -1)
    temp_list.append(cv2.resize(img, (100, 100)))
img_stack = np.hstack(temp_list)
img_total = plot_images(img_stack, 36, 10)
cv2.imwrite("./plots/a13_ch45.png", img_total)

temp_list = []
for r in a15_sorted_entropy:
    img = cv2.imread(r[0].replace("train", "out" ).replace('tif', 'png'), -1)
    temp_list.append(cv2.resize(img, (100, 100)))
img_stack = np.hstack(temp_list)
img_total = plot_images(img_stack, 37, 10)
cv2.imwrite("./plots/a15_ch123.png", img_total)

temp_list = []
for r in a15_sorted_entropy_2:
    img = cv2.imread(r[0].replace("train", "out").replace('tif', 'png'), -1)
    temp_list.append(cv2.resize(img, (100, 100)))
img_stack = np.hstack(temp_list)
img_total = plot_images(img_stack, 37, 10)
cv2.imwrite("./plots/a15_ch45.png", img_total)

True

## Treatment a15 channels 123
![](./plots/a15_ch123.png)

## Control a13 channels 123
![](./plots/a13_ch123.png)

## Treatment a15 channels 45
![](./plots/a15_ch45.png)

## Control a13 channels 45
![](./plots/a13_ch45.png)

We can see:
- Cell size varies within each group.
- There is no obvious morphology difference between treatment `a15` and control `a13`.

## t-SNE Plots

![](./plots/t-sne_c123.png)

![](./plots/t-sne_c45.png)